# ***MODOAP - Détection d'objets dans les images d'un corpus***

Ce notebook permet de détecter et extraire des objets présents dans les images d'un corpus situé sur un Google Drive.

Les objets détectés correspondent aux 80 classes issues du corpus COCO :



```
# class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
```




Le mode GPU est nécessaire (Runtime -> Change Runtime Type -> GPU)

## 0. Connexion à un compte Google Drive et création de l'architecture

Nécessite de se connecter à son compte Google Drive et d'entrer un code de vérification.

Crée un dossier Outils_Modoap sur le Drive qui servira à stocker les poids utilisés et les objets extraits.

In [ ]:
import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")

if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Detection_Objets"):
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Objets/Objets_extraits')
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Objets/Poids')
  os.makedirs('/content/drive/My Drive/Outils_Modoap/Detection_Objets/Logs')

Mounted at /content/drive


## 1. Spécification du corpus

Nécessite d'entrer le chemin absolu vers le dossier contenant le corpus sur le drive. 
La racine du Google Drive est */content/drive/My Drive/*

Possibilité de copier/coller le chemin depuis la fenêtre de gauche : *Files -> clic droit sur un dossier -> Copy Path*

Exemple de chemin:

/content/drive/My Drive/Corpus/corpus_pour_detection/ 

Ou bien entrer "exemple" pour télécharger un corpus de démonstration.

In [ ]:
dossier_corpus = input("Entrer le chemin absolu du dossier contenant le corpus, ou taper \"exemple\" ")
if dossier_corpus == "exemple" :
  print('je télécharge un corpus')
  if not os.path.exists("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection/"):
    os.makedirs('/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection/')
  %cd /content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection/
  !wget https://github.com/MODOAP/corpus_test/releases/download/1/corpus_detection_objets.zip
  !7z x ./corpus_detection_objets.zip
  os.remove("/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection/corpus_detection_objets.zip")
  dossier_corpus = "/content/drive/My Drive/Outils_Modoap/Corpus_demonstrations/corpus_detection/"

## 2. Configuration de l'algorithme

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt


%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install -r requirements.txt
  !python setup.py install
else : print("L'algorithme est déjà téléchargé")

##########################################################

%cd /root/Mask_RCNN
###############################################################
%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5
###############################################################


# Root directory of the project
ROOT_DIR = os.path.abspath("./")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = "/content/drive/My Drive/Outils_Modoap/Detection_Objets/Logs"

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = dossier_corpus

###############################################################

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

###############################################################


# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

###############################################################

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# 3. Detection sur le corpus

Génère le fichier annotations_objets.json dans le dossier contenant le corpus sur le drive.

Ce fichier contient les annotations des objets des images.

Si ce fichier est présent dans le dossier du corpus car il a déjà été généré, cette cellule peut être passée. 

In [ ]:
import glob
import cv2
import json
dossier_corpus = dossier_corpus + "/"
corpus = [image for image in glob.glob(dossier_corpus+"/*.*")]
dicocorpus = {}
print("taille du corpus : ", len(corpus))
for nomfichier in corpus : 
  image = cv2.imread(nomfichier)
  results = model.detect([image], verbose=0)
  r = results[0]
  listeobjets = [vecteur for vecteur in r["rois"]]
  dicoimage={}

  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              class_names, r['scores'])

  for i in range(len(r["rois"])) :
    vecteur_region = [str(coord) for coord in listeobjets[i]]
    classe = class_names[r["class_ids"][i]]
    score = str(r["scores"][i])
    dicoobjet = {"regions":vecteur_region, "classe" : classe, "score":score}
    dicoimage["objet"+str(i)] = dicoobjet
  dicocorpus[nomfichier] = dicoimage

with open(os.path.join(dossier_corpus, "annotations_objets.json"), "w") as dic :
  json.dump(dicocorpus, dic)

## 4. Extraction des objets à partir du fichier d'annotation sur le drive

Les objets extraits sont stockés sur le drive dans /content/drive/My Drive/Outils_Modoap/Detection_Objets/Objets_extraits/ 

In [ ]:
import json
from google.colab.patches import cv2_imshow

with open(os.path.join(dossier_corpus, "annotations_objets.json"), "r") as mondic :
  dicocorpus2 = json.load(mondic)

for nomfichier in dicocorpus2 :
  compteur = 0
  nomfichier2 = str(nomfichier.split("/")[-1])
  image = cv2.imread(nomfichier)
  for objet in dicocorpus2[nomfichier].values():
    y1 = int(objet["regions"][0])
    y2 = int(objet["regions"][2])
    x1 = int(objet["regions"][1])
    x2 = int(objet["regions"][3])
    imagecropped = image[y1:y2, x1:x2] # [Y1:Y2, X1:X2] Le vecteur est sous forme [Y1 X1 Y2 X2]
    filnam = "/content/drive/My Drive/Outils_Modoap/Detection_Objets/Objets_extraits/"+str(nomfichier2)+"_objet"+str(compteur)+"_"+str(objet["classe"])+".jpg"
    print(filnam)
    cv2.imwrite(filnam, imagecropped)
   # cv2_imshow(imagecropped)
    compteur += 1
